# Deezer Dataset Loader

This notebook demonstrates the Deezer Dataset Loader (`sd_bandits.obp_extensions.dataset.DeezerDataset`).

The Deezer data comes with 974,960 users and 862 playlists. Both users and playlists have a 97-dimensional vector of features such that:

$$p(u,i) = \frac{1}{1+\exp(- \langle x_u,\theta_i\rangle)}$$

where $x_u$ is the user vector, $\theta_i$ is the item vector, and $p(u,i)$ is the "ground-truth" probability of the user clicking the item (irrespective of its position).

We want log data that could reasonably be the output of a random experiment with the Deezer data. In other words, if we show each user a completely random set of items, what will they click?

Following the implementation from Deezer's own simulation code ([see here](https://github.com/deezer/carousel_bandits/blob/master/environment.py#L77)), we simulate 100 batches where 20,000 users are selected with replacement. 

Each user is presented with `len_list=12` randomly selected items, and the $p(u,i)$ is calculated for each item. Then each probability is turned into a 0/1 reward outcome by simulating a Bernoulli random variable with $p=p(u,i)$. The rewards are tabulated using Deezer's cascading reward system:
- If the user clicks none of the 12 items, we record the system as receiving 0 reward for each of the `len_init=3` items in the list, because we assume they only saw the first 3 items.
- If the user clicks items $i_1,\dots,i_j$, where $i_1$ is earliest and $i_j$ is latest the list, we record the 0/1 rewards for the first $\max(l_{\textrm{init}},i_j)$ items. **NOTE**: Deezer's code actually records the first $i_j$ rewards even if $i_j<l_{\textrm{init}}$. We checked with Deezer and they acknowledged the simplification, noting it doesn't affect the results appreciably. We chose to follow the description in the paper.
- (Optionally, we can choose to ignore the cascade rule and always record all `len_list` 0/1 rewards.)

The relevant section from the paper is excerpted below:


> Simulations proceed as follows. At each [batch], a random subset of users (20 000, in the following) is presented to
several sequential algorithms a.k.a. policies to be evaluated. These policies must then recommend an ordered set of
L = 12 playlists to each user. Streams, i.e. positive binary rewards, are generated according to the aforementioned
display-to-stream probabilities and to a configurable cascading browsing model capturing that users explore the carousel
from left to right and might not see all recommended playlists. At the end of each [batch], all policies update their model
based on the set of users and on binary rewards received from displayed playlists. Expected cumulative regrets of
policies [2, 22, 39] w.r.t. the optimal top-L playlists sets according to pui probabilities are computed.

> An active user who did not stream any card during the [batch] only saw the Linit first ones.
• An active user who streamed the ith card, with i ∈ {1, ..., L}, saw all cards from ranks 1 to max(Linit,i).

[[Source]](https://arxiv.org/pdf/2009.06546.pdf)

In [1]:
from sd_bandits.obp_extensions.dataset import DeezerDataset
import numpy as np

The `DeezerDataset` class lets you override `len_list` (default 12) and `len_init` (default 3) if desired.

In [2]:
deezer_data = DeezerDataset(
    user_features="../data/deezer_carousel_bandits/user_features.csv",
    playlist_features= "../data/deezer_carousel_bandits/playlist_features.csv"
)

The `obtain_batch_bandit_feedback` method lets you override Deezer's simulation defaults of `n_batches` (100), `users_per_batch` (20,000), `cascade` (True), `replace_within_batch` (True), as well as random `seed`.

In [3]:
feedback_deezer_cascade = deezer_data.obtain_batch_bandit_feedback(
    n_batches=2,
    users_per_batch=500,
    cascade=True, seed=1
)

Generating feedback:: 100%|██████████| 1000/1000 [00:00<00:00, 161549.28it/s]


In [4]:
print("Cascade is enabled, so we see between 3 and 12 items per user per turn")
print("minimum number of actions is thus 2 batches * 500 users * 3 items = 3000")
print("feedback dict:")
for key, value in feedback_deezer_cascade.items():
    if key[0:2] != "n_":
        print(f"  {key}: {type(value)}, {value.shape}")
    else:
        print(f"  {key}: {value}")

Cascade is enabled, so we see between 3 and 12 items per user per turn
minimum number of actions is thus 2 batches * 500 users * 3 items = 3000
feedback dict:
  action: <class 'numpy.ndarray'>, (3354,)
  reward: <class 'numpy.ndarray'>, (3354,)
  position: <class 'numpy.ndarray'>, (3354,)
  context: <class 'numpy.ndarray'>, (3354, 97)
  action_context: <class 'numpy.ndarray'>, (3354, 97)
  pscore: <class 'numpy.ndarray'>, (3354,)
  n_rounds: 3354
  n_actions: 862


In [5]:
feedback_deezer_no_cascade = deezer_data.obtain_batch_bandit_feedback(
    n_batches=2,
    users_per_batch=500,
    cascade=False
)

Generating feedback:: 100%|██████████| 1000/1000 [00:00<00:00, 60200.71it/s]


In [6]:
print("cascade is disabled, so we ways see 12 items per user per turn")
print("number of actions is thus 2 batches * 500 users * 12 items = 12,000")
print("feedback dict:")
for key, value in feedback_deezer_no_cascade.items():
    if key[0:2] != "n_":
        print(f"  {key}: {type(value)}, {value.shape}")
    else:
        print(f"  {key}: {value}")

cascade is disabled, so we ways see 12 items per user per turn
number of actions is thus 2 batches * 500 users * 12 items = 12,000
feedback dict:
  action: <class 'numpy.ndarray'>, (12000,)
  reward: <class 'numpy.ndarray'>, (12000,)
  position: <class 'numpy.ndarray'>, (12000,)
  context: <class 'numpy.ndarray'>, (12000, 97)
  action_context: <class 'numpy.ndarray'>, (12000, 97)
  pscore: <class 'numpy.ndarray'>, (12000,)
  n_rounds: 12000
  n_actions: 862
